# Comparing the different Polymath projects

We start by setting up, loading the urls of the discussions, and putting everything in a frame.

In [1]:
import yaml
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20, 20)

from bs4 import BeautifulSoup
from collections import Counter
import json
import pandas as pd
from pandas import DataFrame
from urllib.parse import urlparse

from comment_thread import *
from author_network import *

In [2]:
def process_polymath(project, split=False):
    message = "Polymath {}".format(project[-1]) if project.startswith("pm") else "Mini Polymath {}".format(project[-1])
    SETTINGS = {
    'msg' : message,
    'filename' : message.replace(" ", ""),
    'source' : project,
    'urls' : [],
    'type' : '',
    'parser' : 'html5lib',
    'cmap' : plt.cm.Paired, # not a string, this is an object
    'vmin' : 1,
    'vmax' : 100}
    
    print("loading data")
    with open("DATA/"+ SETTINGS['source'] + ".txt", "r") as input:
        soup = BeautifulSoup(input.readline(), "html.parser")
    
    items = soup.find("ul").find_all("li")
    urls = [item.find("a").get("href") for item in items]
    titles = [item.text for item in items]
    
    if project == "pm1":
        research = [title.startswith(" (") for title in titles]
    else:
        research = ["Discussion" not in title for title in titles]
        
    pm_frame = DataFrame({
        'url': urls,
        'title' : titles,
        'blog' : [urlparse(url).netloc.split('.')[0].title() for url in urls],
        'research' : research},
        columns = ['title', 'url', 'blog', 'research'])
        
    pm_frame['thread'] = [THREAD_TYPES[blog](url) for (url, blog) in zip(pm_frame['url'], pm_frame['blog'])]
    pm_frame['number of comments'] = pm_frame['thread'].apply(lambda x: len(x.node_name.keys()))
    
    indices = pm_frame.index
    threads = pm_frame.thread
    pm_frame['mthread (single)'] = pm_frame['thread'].apply(MultiCommentThread)
    pm_frame['mthread (accumulated)'] = Series([MultiCommentThread(*threads[0:i+1]) for i in indices],
                                               index=indices)
    pm_frame['network'] = pm_frame['mthread (accumulated)'].apply(AuthorNetwork)
    
    if split and not pm_frame['research'].all():
        r_indices = pm_frame[pm_frame['research']].index
        d_indices = pm_frame[~pm_frame['research']].index
        r_threads = pm_frame[pm_frame['research']].thread
        d_threads = pm_frame[~pm_frame['research']].thread
        pm_frame['r_mthread (accumulated)'] = Series([MultiCommentThread(*r_threads[0:i+1]) for i in r_indices],
                                                 index=r_indices)
        pm_frame['d_mthread (accumulated)'] = Series([MultiCommentThread(*d_threads[0:i+1]) for i in d_indices],
                                                 index=d_indices)
        pm_frame['r_network'] = pm_frame[pm_frame['research']]['r_mthread (accumulated)'].apply(AuthorNetwork)
        pm_frame['d_network'] = pm_frame[~pm_frame['research']]['d_mthread (accumulated)'].apply(AuthorNetwork)
        pm_frame = pm_frame.reindex_axis(['title', 'url', 'blog', 'research', 'number of comments',
                                      'thread', 'mthread (single)',
                                      'mthread (accumulated)', 'network',
                                      'r_mthread (accumulated)', 'r_network',
                                      'd_mthread (accumulated)', 'd_network'],
                                     axis=1)
    else:
        pm_frame = pm_frame.reindex_axis(['title', 'url', 'blog', 'research', 'number of comments',
                                      'thread', 'mthread (single)',
                                      'mthread (accumulated)', 'network'],
                                     axis=1)

    pm_frame.index = pd.MultiIndex.from_tuples([(SETTINGS['msg'], i) for i in indices],
                                               names=['Project', 'Ord'])
    
    return pm_frame

In [3]:
PM3_FRAME = process_polymath("pm3", split=True)
PM3_FRAME

loading data


title  \
Project    Ord                                                      
Polymath 3 0     The polynomial Hirsch conjecture, a proposal ...   
           1     The polynomial Hirsch conjecture, a proposal ...   
           2     The polynomial Hirsch conjecture - how to imp...   
           3     The Polynomial Hirsch Conjecture: Discussion ...   
           4     The Polynomial Hirsch Conjecture: Discussion ...   
           5         Plans for polymath3 (Dec 8, 2009) Inactive.    
           6     The Polynomial Hirsch Conjecture: The Crux of...   
           7     Polynomial Hirsch Conjecture (Sep 29, 2010)  ...   
           8     The Polynomial Hirsch Conjecture 2 (Oct 3, 20...   
           9     Polymath3 : Polynomial Hirsch Conjecture 3 (O...   
           10    Polymath3 : Polynomial Hirsch Conjecture 4 (O...   

                                                              url      blog  \
Project    Ord                                                                
Polymath 3 0    http://gilkalai.wordpress.com/2009/07/17/the-p...  Gilkalai   
           1    http://gilkalai.wordpress.com/2009/07/28/polym...  Gilkalai   
           2    http://gilkalai.wordpress.com/2009/07/30/the-p...  Gilkalai   
           3    http://gilkalai.wordpress.com/2009/08/09/the-p...  Gilkalai   
           4    http://gilkalai.wordpress.com/2009/10/06/the-p...  Gilkalai   
           5    http://gilkalai.wordpress.com/2009/12/08/plans...  Gilkalai   
           6    http://gilkalai.wordpress.com/2010/06/19/the-p...  Gilkalai   
           7    http://gilkalai.wordpress.com/2010/09/29/polym...  Gilkalai   
           8    http://gilkalai.wordpress.com/2010/10/03/polym...  Gilkalai   
           9    http://gilkalai.wordpress.com/2010/10/10/polym...  Gilkalai   
           10   http://gilkalai.wordpress.com/2010/10/21/polym...  Gilkalai   

               research  number of comments  \
Project    Ord                                
Polymath 3 0       True                  31   
           1       True                   2   
           2       True                  11   
           3      False                 109   
           4      False                  13   
           5       True                   2   
           6       True                   3   
           7       True                 112   
           8       True                 103   
           9       True                  98   
           10      True                  69   

                                                           thread  \
Project    Ord                                                      
Polymath 3 0    <comment_thread.CommentThreadGilkalai object a...   
           1    <comment_thread.CommentThreadGilkalai object a...   
           2    <comment_thread.CommentThreadGilkalai object a...   
           3    <comment_thread.CommentThreadGilkalai object a...   
           4    <comment_thread.CommentThreadGilkalai object a...   
           5    <comment_thread.CommentThreadGilkalai object a...   
           6    <comment_thread.CommentThreadGilkalai object a...   
           7    <comment_thread.CommentThreadGilkalai object a...   
           8    <comment_thread.CommentThreadGilkalai object a...   
           9    <comment_thread.CommentThreadGilkalai object a...   
           10   <comment_thread.CommentThreadGilkalai object a...   

                                                 mthread (single)  \
Project    Ord                                                      
Polymath 3 0    <comment_thread.MultiCommentThread object at 0...   
           1    <comment_thread.MultiCommentThread object at 0...   
           2    <comment_thread.MultiCommentThread object at 0...   
           3    <comment_thread.MultiCommentThread object at 0...   
           4    <comment_thread.MultiCommentThread object at 0...   
           5    <comment_thread.MultiCommentThread object at 0...   
           6    <comment_thread.MultiCommentThread object at 0...   
   

In [4]:
PM4_FRAME = process_polymath("pm4", split=True)
PM4_FRAME

loading data


title  \
Project    Ord                                                      
Polymath 4 0     Proposal for the project and initial research...   
           1         Research thread II Opened Aug 9.  Inactive.    
           2       Research thread III Opened Aug 13.  Inactive.    
           3        Research thread IV Opened Aug 28.  Inactive.    
           4           Research thread V Opened Oct 27.  Active.    
           5      Discussion thread I Opened July 28.  Inactive.    
           6        Discussion thread II Opened Jun 29.  Active.    

                                                              url  \
Project    Ord                                                      
Polymath 4 0    http://polymathprojects.org/2009/07/27/proposa...   
           1    http://polymathprojects.org/2009/08/09/researc...   
           2    http://polymathprojects.org/2009/08/13/researc...   
           3    http://polymathprojects.org/2009/08/28/researc...   
           4    http://polymathprojects.org/2009/10/27/researc...   
           5    http://polymathprojects.org/2009/07/28/determi...   
           6    http://polymathprojects.org/2010/06/29/draft-v...   

                            blog research  number of comments  \
Project    Ord                                                  
Polymath 4 0    Polymathprojects     True                 130   
           1    Polymathprojects     True                 143   
           2    Polymathprojects     True                 114   
           3    Polymathprojects     True                  64   
           4    Polymathprojects     True                  40   
           5    Polymathprojects    False                  52   
           6    Polymathprojects    False                  31   

                                                           thread  \
Project    Ord                                                      
Polymath 4 0    <comment_thread.CommentThreadPolymath object a...   
           1    <comment_thread.CommentThreadPolymath object a...   
           2    <comment_thread.CommentThreadPolymath object a...   
           3    <comment_thread.CommentThreadPolymath object a...   
           4    <comment_thread.CommentThreadPolymath object a...   
           5    <comment_thread.CommentThreadPolymath object a...   
           6    <comment_thread.CommentThreadPolymath object a...   

                                                 mthread (single)  \
Project    Ord                                                      
Polymath 4 0    <comment_thread.MultiCommentThread object at 0...   
           1    <comment_thread.MultiCommentThread object at 0...   
           2    <comment_thread.MultiCommentThread object at 0...   
           3    <comment_thread.MultiCommentThread object at 0...   
           4    <comment_thread.MultiCommentThread object at 0...   
           5    <comment_thread.MultiCommentThread object at 0...   
           6    <comment_thread.MultiCommentThread object at 0...   

                                            mthread (accumulated)  \
Project    Ord                                                      
Polymath 4 0    <comment_thread.MultiCommentThread object at 0...   
           1    <comment_thread.MultiCommentThread object at 0...   
           2    <comment_thread.MultiCommentThread object at 0...   
           3    <comment_thread.MultiCommentThread object at 0...   
           4    <comment_thread.MultiCommentThread object at 0...   
           5    <comment_thread.MultiCommentThread object at 0...   
           6    <comment_thread.MultiCommentThread object at 0...   

                                                          network  \
Project    Ord                                                      
Polymath 4 0    <author_network.AuthorNetwork object at 0x112c...   
           1    <author_network.AuthorNetwork object at 0x112c...   
           2    <author_network.AuthorNetwork object at 0x112c...   
           3    <au

In [5]:
PM5_FRAME = process_polymath("pm5", split=True)
PM5_FRAME

loading data


title  \
Project    Ord                                                      
Polymath 5 0             Zeroth Post (Dec 17 2009 - Jan 6 2010).    
           1                        First Post (Jan 6 - Jan 12).    
           2                       Second Post (Jan 9 - Jan 11).    
           3                       Third Post (Jan 11 - Jan 14).    
           4                          Fourth Post (Jan 14 - 16).    
           5                             Fifth Post (Jan 16-19).    
           6                  First Theoretical Post (Jan 19-21)    
           7                 Second Theoretical Post (Jan 21-26)    
           8                  Third Theoretical Post (Jan 26 -?)    
           9             Fourth Theoretical Post (Jan 30- Feb 2)    
           10             Fifth Theoretical Post (Feb 2 - Feb 5)    
           11              Sixth Theoretical Post (Feb 5- Feb 8)    
           12              Seventh Theoretical Post (Feb 8 - 19)    
           13            Eighth Theoretical Post (Feb 19-Feb 24)    
           14              Ninth Theoretical Most (Feb 24-Mar 2)    
           15               Tenth Theoretical Post (Mar 2-Mar 7)    
           16           Eleventh Theoretical Post (Mar 7-Mar 12)    
           17           Twelfth Theoretical Post (Mar 13-Mar 22)    
           18        Thirteenth Theoretical Post (Mar 23-Apr 24)    
           19        Fourteenth Theoretical Post (Apr 25-Jun 20)    
           20          Fifteenth Theoretical Post (Jun 21-Jul 3)    
           21          Sixteenth theoretical Post (Jul 4-Jul 17)    
           22        Seventeenth Theoretical Post (Jul 18-Sep 2)    
           23          Eighteenth Theoretical Post (Sep 3-Sep 5)    
           24          Nineteenth Theoretical Post (Sep 6-Sep 9)    
           25         Twentieth Theoretical Post (Sep 10-Sep 20)    
           26   Twenty-first Theoretical Post (Sep 21 - Aug 21...   
           27                        A possible revival (Aug 22)    
           28        First guest post from Gil Kalai (Aug 22-26)    
           29       Second guest post from Gil Kalai (Aug 27-30)    
           30   An attempt to get back into the diagonal decom...   
           31      Third guest post from Gil Kalai (Sep 4-Sep 5)    
           32               Three generalizations (Sep 6-Sep 18)    
           33   The modular version of Roth's AP discrepancy t...   

                                                              url    blog  \
Project    Ord                                                              
Polymath 5 0    http://gowers.wordpress.com/2009/12/17/erdoss-...  Gowers   
           1    http://gowers.wordpress.com/2010/01/06/erdss-d...  Gowers   
           2    http://gowers.wordpress.com/2010/01/09/erds-di...  Gowers   
           3    http://gowers.wordpress.com/2010/01/11/the-erd...  Gowers   
           4    http://gowers.wordpress.com/2010/01/14/the-erd...  Gowers   
           5    http://gowers.wordpress.com/2010/01/16/the-erd...  Gowers   
           6    http://gowers.wordpress.com/2010/01/19/edp1-th...  Gowers   
           7    http://gowers.wordpress.com/2010/01/21/edp2-a-...  Gowers   
           8    http://gowers.wordpress.com/2010/01/26/edp3-a-...  Gowers   
           9    http://gowers.wordpress.com/2010/01/30/edp4-fo...  Gowers   
           10   http://gowers.wordpress.com/2010/02/02/edp5-an...  Gowers   
           11   http://gowers.wordpress.com/2010/02/05/edp6-wh...  Gowers   
           12   http://gowers.wordpress.com/2010/02/08/edp7-em...  Gowers   
           13   http://gowers.wordpress.com/2010/02/19/edp8-wh...  Gowers   
           14   http://gowers.wordpress.com/2010/02/24/edp9-a-...  Gowers   
           15   http://gowers.wordpress.com/2010/03/02/edp10-a...  Gowers   
           16   http://gowers.wordpress.com/2010/03/07/edp11-t...  Gowers   
           17   http://gowers.wordpress.com/2010/03/13/edp12-r...  Gowers   
           18   http://gowers.wordp

In [6]:
PM6_FRAME = process_polymath("pm6", split=True)
PM6_FRAME

loading data


,,title,url,blog,research,number of comments,thread,mthread (single),mthread (accumulated),network
Project,Ord,,,,,,,,,
Polymath 6,0,Blog post by Tom Sanders about the ideas and d...,http://polymathprojects.org/2011/02/05/polymat...,Polymathprojects,True,16,<comment_thread.CommentThreadPolymath object a...,<comment_thread.MultiCommentThread object at 0...,<comment_thread.MultiCommentThread object at 0...,<author_network.AuthorNetwork object at 0x129e...


In [7]:
PM7_FRAME = process_polymath("pm7", split=True)
PM7_FRAME

loading data


title  \
Project    Ord                                                      
Polymath 7 0      First research thread (June 3, 2012)  Inactive    
           1     Current discussion thread (June 9, 2012) Active    
           2          Research thread 1 (June 12, 2012) Inactive    
           3          Research thread 2 (June 15, 2012) Inactive    
           4          Research thread 3 (June 24, 2012) Inactive    
           5     Research thread 4 (September 10, 2012) Inactive    
           6                  Research thread 5 (August 9, 2013)    

                                                              url  \
Project    Ord                                                      
Polymath 7 0    http://polymathprojects.org/2012/06/03/polymat...   
           1    http://polymathprojects.org/2012/06/09/polymat...   
           2    http://polymathprojects.org/2012/06/12/polymat...   
           3    http://polymathprojects.org/2012/06/15/polymat...   
           4    http://polymathprojects.org/2012/06/24/polymat...   
           5    http://polymathprojects.org/2012/09/10/polymat...   
           6    http://polymathprojects.org/2013/08/09/polymat...   

                            blog research  number of comments  \
Project    Ord                                                  
Polymath 7 0    Polymathprojects     True                 102   
           1    Polymathprojects     True                  40   
           2    Polymathprojects     True                  89   
           3    Polymathprojects     True                  95   
           4    Polymathprojects     True                  97   
           5    Polymathprojects     True                 102   
           6    Polymathprojects     True                   5   

                                                           thread  \
Project    Ord                                                      
Polymath 7 0    <comment_thread.CommentThreadPolymath object a...   
           1    <comment_thread.CommentThreadPolymath object a...   
           2    <comment_thread.CommentThreadPolymath object a...   
           3    <comment_thread.CommentThreadPolymath object a...   
           4    <comment_thread.CommentThreadPolymath object a...   
           5    <comment_thread.CommentThreadPolymath object a...   
           6    <comment_thread.CommentThreadPolymath object a...   

                                                 mthread (single)  \
Project    Ord                                                      
Polymath 7 0    <comment_thread.MultiCommentThread object at 0...   
           1    <comment_thread.MultiCommentThread object at 0...   
           2    <comment_thread.MultiCommentThread object at 0...   
           3    <comment_thread.MultiCommentThread object at 0...   
           4    <comment_thread.MultiCommentThread object at 0...   
           5    <comment_thread.MultiCommentThread object at 0...   
           6    <comment_thread.MultiCommentThread object at 0...   

                                            mthread (accumulated)  \
Project    Ord                                                      
Polymath 7 0    <comment_thread.MultiCommentThread object at 0...   
           1    <comment_thread.MultiCommentThread object at 0...   
           2    <comment_thread.MultiCommentThread object at 0...   
           3    <comment_thread.MultiCommentThread object at 0...   
           4    <comment_thread.MultiCommentThread object at 0...   
           5    <comment_thread.MultiCommentThread object at 0...   
           6    <comment_thread.MultiCommentThread object at 0...   

                                                          network  
Project    Ord                                                     
Polymath 7 0    <author_network.AuthorNetwork object at 0x1376...  
           1    <author_network.AuthorNetwork object at 0x1376...  
           2    <author_network.AuthorNetwork object at 0x129e...  
           3    <author_

In [9]:
PM8_FRAME = process_polymath("pm8", split=True)
PM8_FRAME

loading data


KeyError: 'Sbseminar'

In [ ]:
PM3_FRAME['network'].iloc[-1].draw_graph(title="Author network for {}".format("Polymath 3"))

In [ ]:
PM3_FRAME['network'].iloc[-1].plot_author_activity_pie(project="Polymath 3")
PM3_FRAME['network'].iloc[-1].plot_degree_centrality(project="Polymath 3")

In [ ]:
PM3_FRAME['mthread (accumulated)'].iloc[-1].plot_activity('thread', intervals=1, max_span=timedelta(500),
                                                         project="Polymath 3")
PM3_FRAME['mthread (accumulated)'].iloc[-1].plot_growth(drop_last=2,
                                                         project="Polymath 3")

In [ ]:
PM4_FRAME['network'].iloc[-1].draw_graph(title="Author network for {}".format("Polymath 4"))

In [ ]:
PM4_FRAME['network'].iloc[-1].plot_author_activity_pie(project="Polymath 4")
PM4_FRAME['network'].iloc[-1].plot_degree_centrality(project="Polymath 4")

In [ ]:
PM4_FRAME['mthread (accumulated)'].iloc[-1].plot_activity('thread', intervals=1, max_span=timedelta(800),
                                                         project="Polymath 4")
PM4_FRAME['mthread (accumulated)'].iloc[-1].plot_growth(drop_last=15,
                                                         project="Polymath 4")

In [ ]:
PM5_FRAME['network'].iloc[-1].draw_graph(title="Author network for {}".format("Polymath 5"))

In [ ]:
PM5_FRAME['network'].iloc[-1].plot_author_activity_pie(project="Polymath 5")
PM5_FRAME['network'].iloc[-1].plot_degree_centrality(project="Polymath 5")

In [ ]:
PM5_FRAME['mthread (accumulated)'].iloc[-1].plot_activity('thread', intervals=1, max_span=timedelta(1500),
                                                         project="Polymath 5")
PM5_FRAME['mthread (accumulated)'].iloc[-1].plot_growth(drop_last=0,
                                                         project="Polymath 5")

In [ ]:
PM7_FRAME['network'].iloc[-1].draw_graph(title="Author network for {}".format("Polymath 7"))

In [ ]:
PM7_FRAME['network'].iloc[-1].plot_author_activity_pie(project="Polymath 7")
PM7_FRAME['network'].iloc[-1].plot_degree_centrality(project="Polymath 7")

In [ ]:
PM7_FRAME['mthread (accumulated)'].iloc[-1].plot_activity('thread', intervals=1, max_span=timedelta(1500),
                                                         project="Polymath 7")
PM7_FRAME['mthread (accumulated)'].iloc[-1].plot_growth(drop_last=0,
                                                         project="Polymath 7")